In [1]:
import torch
import numpy as np
import scipy.io as sio
import os
import time

In [2]:
torch.cuda.is_available()
t=torch.cuda.FloatTensor([2,3])

import gc

g=gc.collect()

In [3]:
class EnsemblePursuitPyTorch():
    
    def calculate_cost_delta(self):
        #print(np.dot(self.current_v.T,self.current_v))
        #cost_delta=np.clip(self.current_v.T@self.X,0,None)/np.dot(self.current_v.T,self.current_v)[0]-self.lambd
        #print(torch.matmul(self.current_v,self.X).size())
        cost_delta=torch.clamp(torch.matmul(self.current_v,self.X),min=0,max=None)/torch.matmul(self.current_v,self.current_v)-self.lambd
        return cost_delta
    
    def fit_one_assembly(self):
        with torch.cuda.device(0) as device:
            choose_neuron_idx=np.random.randint(0,self.X.size()[1],1)[0]
            self.current_u=torch.zeros([self.X.size(1)]).cuda()
            self.current_u[choose_neuron_idx]=1
            self.current_v=self.X[:,choose_neuron_idx]
            #print(self.current_v.size())
            max_delta_cost=1000
            self.i=0
            while max_delta_cost>0:
                cost_delta=self.calculate_cost_delta()
                #print(cost_delta.size())
                #print(self.current_u.size())
                mask=self.current_u.clone()
                mask[self.current_u==0]=1
                mask[self.current_u!=0]=0
                masked_cost_delta=mask*cost_delta
                #print(masked_cost_delta.type())
                #print(masked_cost_delta)
                values,sorted_neurons=masked_cost_delta.sort()
                max_delta_neuron=sorted_neurons[-1]
                #print(max_delta_neuron.item())
                #print(values)
                max_delta_cost=values[-1]
                if max_delta_cost>0:
                    self.current_v=(self.current_v+self.X[:,max_delta_neuron.item()])/2
                    self.current_u[max_delta_neuron.item()]=1
                self.i+=1
            #print(i)
            self.U=torch.cat((self.U,self.current_u.view(self.X.size(1),1)),1)
            self.V=torch.cat((self.V,self.current_v.view(1,self.X.size(0))),0)
    
    
    
    def fit_transform(self,X,lambd,n_ensembles=None):
        with torch.cuda.device(0) as device:
            self.lambd=lambd
            self.X=torch.cuda.FloatTensor(X)
            self.U=torch.zeros((self.X.size(1),1)).cuda()
            self.V=torch.zeros([1,self.X.size(0)]).cuda()
            self.nr_of_neurons=[]
            #self.current_u=torch.zeros([self.X.size(1)]).cuda()
            for iteration in range(0,n_ensembles):
                self.fit_one_assembly()
                self.nr_of_neurons.append(self.i)
                U_V=torch.mm(self.current_u.view(self.X.size(1),1),self.current_v.view(1,self.X.size(0)))
                #print(U_V.size())
                self.X=(self.X.t()-U_V).t()
                self.X[self.X<0]=0
                #Reset u for new iteration
            self.U=self.U[:,1:]
            self.V=self.V[1:,:]
            print(self.X.size())
            print(self.U.size())
            print(self.V.size())
            return torch.matmul(self.U,self.V).t().cpu(), self.nr_of_neurons, self.U.cpu(), self.V.cpu()

In [4]:
X=sio.loadmat('/home/maria/Documents/EnsemblePursuit/data/natimg2800_M170717_MP034_2017-09-11.mat')['stim']['resp'][0][0]
X[X<0]=0
print(X.shape)

(5880, 10103)


In [ ]:
np.random.seed(7)
ep=EnsemblePursuitPyTorch()
s=time.time()
U_V,nr_of_neurons,U,V=ep.fit_transform(X,0.35,1000)
e=time.time()
print(e-s)
print(nr_of_neurons)